### Gurobi Implementation

In [1]:
### imports
import numpy as np
import gurobipy as gp
from gurobipy import GRB
from matplotlib import pyplot as plt

#### Set up model parameters

In [ ]:
# define decision variables 
num_programs = XXX
num_counties = XXX
counties = range(num_counties)
programs = range(num_programs)

# parameters--rates of outcomes of interest per county
# perhaps formatted as a matrix of row x col = number of counties x number of programs
#rates = np.array([67,num_programs])
rates = XXX

# equity constraint per county
# equity_cap = XXX

# parameter weights (from regression output)
program_weights = XXX
weight_direction = XXX # (-1 if we want less of an outcome, 1 if we want more of an outcome, for each program)



#### Model setup

In [ ]:
# initialize model
model = gp.Model()

# decision variables
X = model.addVars(counties, programs)

# objective function
model.setObjective(sum(sum(program_weights[j]*((1+(weight_direction[j]*X[i,j]))*rates[i,j]) for j in programs) for i in counties))

model.modelSense = GRB.MINIMIZE

# constraints
model.addConstr(sum(X[i,j] for j in programs) <= 1) # program resource allocation constraint

for i in counties:
    #model.addConstr(sum(program_weights[j]*rates[i,j] for j in programs) - 
    #   sum(program_weights[j]*((1+(weight_direction[j]*X[i,j]))*rates[i,j]) for j in programs) >= equity_cap[i]) # equity constraint
    
    for j in programs: 
        model.addConstr(X[i,j] >= 0) # non-negativity
        model.addConstr(((1+(weight_direction[j]*X[i,j]))*rates[i,j]) >= 0) # ensure rates are between [0,1]
        model.addConstr(((1+(weight_direction[j]*X[i,j]))*rates[i,j]) <= 1) # ensure rates are between [0,1]

# optimize
model.optimize()

In [ ]:
# print allocations for first 10 counties as a check

for i in range(10):
    print('\nCounty ', i, ' allocations:')
    for j in programs: 
        print('Program ', j, ': ', X[i,j].x)